In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

In [ ]:
import logging
import warnings

warnings.simplefilter(action="ignore")
logging.getLogger('tensorflow').disabled = True

In [ ]:
import autograd.numpy as np
import matplotlib.pyplot as plt

import analysis.analysis_picker as ap
from data.dataset import Dataset
import utils.data_processing as dp

### Importing Module

I packaged everything into a module `schematic_utils` with a bunch of sub-modules.
The below assumes this is in an adjacent folder to the `cwd`, if it's not in the path.
Adjust as needed.

These utilities depend on `data.dataset` and `dp`, so make sure those are importable.

In [ ]:
try:
    import schematic_utils
except ImportError:
    import sys
    sys.path.append("../schematic_figure/")
    import schematic_utils

In [ ]:
figsize = (16, 16)
fontsize = 20
dpi = 200

# Figure 6ab - Toy Attacks

Attacks on the toy networks are done using the `schematic_utils.toy` sub-module, which also includes plotting sub-modules.

### `compute_panel_info`

Given a function to compute a neuron's forward pass plus a few parameters, the below function
computes contours and applies a simple gradient-following attack. 

In [ ]:
def compute_panel_info(neuron_fwd_pass, attack_start_coords, N_contours, contour_eps):

    xlims = [0., 0.27]
    ylims = [-0.25, 0.27]
    xN, yN = 20, 26

    activation_contours, contour_vals = \
        schematic_utils.toy.euler.calculate_activation_contours(
            neuron_fwd_pass,
            N=N_contours, contour_eps=contour_eps,
            mn=0.1, mx=0.22,
            contour_lims=[-0.25, 0.3])

    grads, grad_coords = schematic_utils.toy.util.compute_grads(neuron_fwd_pass, xlims, ylims, xN, yN)

    attack = schematic_utils.toy.util.follow_grad(
        np.array(attack_start_coords), neuron_fwd_pass, [-0.1, 0.26], 1e-4)

    attack_activations = [neuron_fwd_pass(point) for point in attack]
    
    return activation_contours, contour_vals, attack, attack_activations, grads, grad_coords

### Define Models

In [ ]:
example_dict = schematic_utils.toy.util.normalize_dict(np.asarray(
    [[1, 0],
     [1, 1],
     [1, -1],
    ]
    ).T)

LCA = schematic_utils.toy.models.LCA(example_dict)
lca_neuron_1_forward_pass = lambda x: \
    LCA.forward_pass(x, num_iters=50)[0]

MLP = schematic_utils.toy.models.MLP([example_dict.T])

mlp_neuron_1_forward_pass = lambda x: \
    MLP.forward_pass(x)[0]

### Run Attacks and Compute Contours

The LCA computations take up to a few minutes.

In [ ]:
lca_N_contours, lca_contour_eps = 5, 1e-2
lca_attack_start_coords = [0.1, 0.05]

lca_activation_contours, lca_contour_vals, lca_attack, lca_attack_activations, lca_grads, lca_grad_coords =\
    compute_panel_info(lca_neuron_1_forward_pass, lca_attack_start_coords, lca_N_contours, lca_contour_eps)

In [ ]:
mlp_N_contours, mlp_contour_eps = 6, 1e-2
mlp_attack_start_coords = [0.1, 0.15]

mlp_activation_contours, mlp_contour_vals, mlp_attack, mlp_attack_activations, mlp_grads, mlp_grad_coords =\
    compute_panel_info(mlp_neuron_1_forward_pass, mlp_attack_start_coords, mlp_N_contours, mlp_contour_eps)

### Make Plots

In [ ]:
f, axs = plt.subplots(ncols=2, figsize=(16, 12))

lca_ax, mlp_ax = axs

schematic_utils.toy.plot.make_contour_panel(
    lca_activation_contours,
    lca_contour_vals,
    lca_attack, lca_attack_activations,
    grads = lca_grads,
    grad_coords = lca_grad_coords,
    weight_vectors=LCA.dictionary[:, :2].T / 6,
    ax=lca_ax)

schematic_utils.toy.plot.make_contour_panel(
    mlp_activation_contours,
    mlp_contour_vals,
    mlp_attack, mlp_attack_activations,
    grads = mlp_grads,
    grad_coords = mlp_grad_coords,
    weight_vectors=LCA.dictionary[:, :2].T / 6,
    ax=mlp_ax,
    skip_x_axis_grads=False);

# Figure 6c - Real Attack

Attacks on the real network are run using the `schematic_utils.real` sub-module. Again, plotting is delefated to a sub-module.

### Building Network

In [ ]:
class slp_lca_params(object):
    def __init__(self):
        self.model_type = "lca"
        self.model_name = "slp_lca_768_latent_cosyne_mnist"
        self.display_name = "LCA"
        self.version = "0.0"
        self.save_info = "test_kurakin_targeted"
        self.overwrite_analysis_log = False

In [ ]:
params_list = [slp_lca_params()]

for params in params_list:
    params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+params.model_name)

analyzer_list = [ap.get_analyzer(params.model_type) for params in params_list]

for analyzer, params in zip(analyzer_list, params_list):
    analyzer.setup(params)
    analyzer.model.setup(analyzer.model_params)
    analyzer.load_analysis(save_info=params.save_info)
    analyzer.model_name = params.model_name

The below was necessary on my machine because the `cp_loc` of the analyzer was not being set correctly.

In [ ]:
root_model_dir = os.path.expanduser("~")+"/Work/Projects/slp_lca_768_latent_cosyne_mnist/"
analysis_dir = root_model_dir + "analysis/0.0/"

analyzer.analysis_params.cp_loc = root_model_dir + "/checkpoints/slp_lca_768_latent_cosyne_mnist_v0.0-100000"

### Loading Attack and Basis Functions

In [ ]:
adversary_images_npz = "class_adversary_images_analysis_test_kurakin_targeted.npz"

savefiles_dir = os.path.expanduser("~")+"/Work/Projects/slp_lca_768_latent_cosyne_mnist/analysis/0.0/savefiles"

npz_filepath = os.path.join(savefiles_dir, adversary_images_npz)

attack = schematic_utils.real.util.retrieve_attack(npz_filepath)

orig_class_bf, attack_class_bf = schematic_utils.real.util.retrieve_basis_functions(analyzer)

In [ ]:
save = False

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 12))

_, ax = schematic_utils.real.plot.make_panel(analyzer, attack, orig_class_bf, attack_class_bf, ax=ax)

if save:  #default is false
    f.savefig("../Figure 6/Figure 6b.svg")

# Figure 6 - Combined

In [ ]:
f = plt.figure(figsize=(2*figsize[0],figsize[1]), dpi=dpi)
fig_shape = (1, 4)

mlp_ax = plt.subplot2grid(fig_shape, loc=(0, 0), colspan=1, fig=f)
lca_ax = plt.subplot2grid(fig_shape, loc=(0, 1), colspan=1, fig=f)
real_ax = plt.subplot2grid(fig_shape, loc=(0, 2), colspan=2, fig=f)

schematic_utils.toy.plot.make_contour_panel(
    mlp_activation_contours,
    mlp_contour_vals,
    mlp_attack, mlp_attack_activations,
    grads = mlp_grads,
    grad_coords = mlp_grad_coords,
    weight_vectors=LCA.dictionary[:, :2].T / 6,
    ax=mlp_ax,
    skip_x_axis_grads=False);

schematic_utils.toy.plot.make_contour_panel(
    lca_activation_contours,
    lca_contour_vals,
    lca_attack, lca_attack_activations,
    grads = lca_grads,
    grad_coords = lca_grad_coords,
    weight_vectors=LCA.dictionary[:, :2].T / 6,
    ax=lca_ax)

for ax in [mlp_ax, lca_ax]:
  for ax_loc in ["top", "bottom", "left", "right"]:
    ax.spines[ax_loc].set_color(None)


schematic_utils.real.plot.make_panel(analyzer, attack, orig_class_bf, attack_class_bf, ax=real_ax);

In [ ]:
for ext in [".png", ".eps"]:
  save_name = (analyzer.analysis_out_dir+"/vis/contours_and_gradients_schematic"
    +"_"+analyzer.analysis_params.save_info+ext)
  f.savefig(save_name, transparent=False, bbox_inches="tight", pad_inches=0.01, dpi=dpi)